In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
from time import time
from QAP_Heuristic import QAP_Heuristic

In [2]:
class TabuSearch(QAP_Heuristic):
    def __init__(self, w, d, tabu_size=100) -> None:
        super().__init__(w, d)
        self.tabu_size = tabu_size

    def __str__(self):
        return "TabuSearch"

    # auxiliary methods
    def generate_neighbors(self, solution):
        neighbors = []
        for i in range(len(solution)):
            for j in range(i+1, len(solution)):
                neighbor = solution.copy()
                neighbor[i], neighbor[j] = solution[j], solution[i]
                neighbors.append(neighbor)
        return neighbors

    # implements the solve method via the Tabu Search algorithm
    def solve(self, n_iter):
        
        best_solution = np.random.permutation(self.n).tolist()  # the .tolist() is janky, but it doesnt work with out it for some reason
        best_cost = self.cost(best_solution)
        
        current_solution = best_solution
        current_cost = best_cost
        
        tabu_list = []

        for _ in range(n_iter):

            if time() > self.MAX_CPU_TIME: break

            neighbors = self.generate_neighbors(current_solution)
            best_neighbor = None
            best_neighbor_cost = float('inf')
            
            for neighbor in neighbors:
                if neighbor not in tabu_list:
                    cost = self.cost(neighbor)
                    if cost < best_neighbor_cost:
                        best_neighbor_cost = cost
                        best_neighbor = neighbor
            
            if best_neighbor:
                tabu_list.append(best_neighbor)
                if len(tabu_list) > self.tabu_size:
                    tabu_list.pop(0)
                
                current_solution = best_neighbor
                current_cost = best_neighbor_cost
                
                if current_cost < best_cost:
                    best_cost = current_cost
                    best_solution = current_solution

        return best_solution, best_cost

    # greedy constructive heuristic to formulate an initial solution
    def greedy_qap(self):
        F, D = self.W.copy(), self.D.copy()
        
        num_facilities = F.shape[0]
        remaining_facilities = set(range(num_facilities))
        remaining_locations = set(range(num_facilities))
        perm = [-1 for _ in range(num_facilities)]
        
        for i in range(num_facilities):
            best_cost = float('inf')
            best_facility, best_location = -1, -1
            
            for f in remaining_facilities:
                for l in remaining_locations:
                    current_cost = self.cost(perm)
                    
                    if current_cost < best_cost:
                        best_cost = current_cost
                        best_facility, best_location = f, l
                        
            perm[best_location] = best_facility
            remaining_facilities.remove(best_facility)
            remaining_locations.remove(best_location)
        
        return perm

In [13]:
# set up params for plots
SMALL, MED, LARGE, LW = 18, 24, 30, 3
plt.rc('axes', titlesize=SMALL)    # fontsize of the axes title
plt.rc('axes', labelsize=SMALL)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL) # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL) # fontsize of the tick labels
plt.rc('legend', fontsize=MED)   # legend fontsize
plt.rc('font', size=SMALL)       # controls default text sizes

In [3]:
import csv

instance_path = '../QAPInstances/'
soln_path     = '../QAPSolns/'

# for managing file opening and closing
def read_integers(filename):
    with open(filename) as f:
        return [int(elem) for elem in f.read().split()]


def open_solution(filename: str):
    file_it = iter(read_integers(filename))
    _ = next(file_it)    # this is just how the files within the lib are formatted
    return next(file_it)

        

def test_hueristic(n_iters=10_000, tai_only=False, bur_only=False):
    """
    n_iters: number of iterations on each test instance
    tai_only: if True, restricts testing to the Tai instances, reducing computation time
    """
    results_filename = '../results/Tabu.csv'
    if bur_only: results_filename = '../results/Tabu-bur.csv'
    if tai_only: results_filename = '../results/Tabu-tai.csv'

    with open(results_filename, mode='w') as f:
        
        writer = csv.writer(f)

        for filename in os.listdir(instance_path):
            
            # skips any instances that are not Tai
            if tai_only and 'tai' not in filename: continue  
            if bur_only and 'bur' not in filename: continue  

            file_it = iter(read_integers(instance_path+filename))

            # open QAP instance param's 
            n = next(file_it)
            w = np.array([[next(file_it) for j in range(n)] for i in range(n)])
            d = np.array([[next(file_it) for j in range(n)] for i in range(n)])

            # generate an instance
            heuristic = TabuSearch(w, d)

            # open up corresponding soln from QAPLib: 
            soln_file = filename[:-4]+'.sln' # this removes the .dat from filename
            
            try:
                qap_soln = open_solution(soln_path+soln_file)
                huerstic_soln, _ = heuristic.solve(n_iters)

                # compute gap
                gap = 100*(heuristic.cost(huerstic_soln) - qap_soln)/qap_soln
                
                writer.writerow([filename,gap])
                print(filename, gap)
                
            # any instances without corresponding solution files are deleted
            except FileNotFoundError:
                os.remove(instance_path+filename) 
    return

In [4]:
test_hueristic(n_iters=100_000, bur_only=True)

TypeError: Can't instantiate abstract class TabuSearch with abstract method __str__